In [18]:
# Importing packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re

from sklearn import linear_model, metrics
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score


from sklearn.feature_selection import RFE

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import scale
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline

import warnings # supress warnings
warnings.filterwarnings('ignore')

import os
#Setting option to diaply maximum rows and columns
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

In [19]:
df_house = pd.read_csv("train.csv")
df_house.head()
train=df_house.copy()
train.shape

(1460, 81)

In [20]:
target=train.pop('SalePrice')

In [21]:
df_test=pd.read_csv('test.csv')
test=df_test.copy()
test.shape

(1459, 80)

In [22]:
dataset = pd.concat(objs=[train, test], axis=0)
dataset.shape

#dataset = pd.get_dummies(dataset)


(2919, 80)

In [23]:
dataset.dtypes

Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
Street            object
Alley             object
LotShape          object
LandContour       object
Utilities         object
LotConfig         object
LandSlope         object
Neighborhood      object
Condition1        object
Condition2        object
BldgType          object
HouseStyle        object
OverallQual        int64
OverallCond        int64
YearBuilt          int64
YearRemodAdd       int64
RoofStyle         object
RoofMatl          object
Exterior1st       object
Exterior2nd       object
MasVnrType        object
MasVnrArea       float64
ExterQual         object
ExterCond         object
Foundation        object
BsmtQual          object
BsmtCond          object
BsmtExposure      object
BsmtFinType1      object
BsmtFinSF1       float64
BsmtFinType2      object
BsmtFinSF2       float64
BsmtUnfSF        float64
TotalBsmtSF      float64
Heating           object


In [37]:
df_house_price=dataset.copy()

In [38]:
for col in df_house_price.columns.values:
        missing_values = df_house_price[col].isnull().sum()

        # drop column if there is more than 70 percent missing values
        if missing_values > len(df_house_price[col])* 70/100:
            print("droping column: {}".format(col))
            df_house_price.drop(col, axis = 1,inplace = True)

droping column: Alley
droping column: PoolQC
droping column: Fence
droping column: MiscFeature


In [89]:
print('Viewing the unique values in the data with counts in descending order')
print()
def check_valuecount():
    for col in df_house_price:
        print(col + " value counts: ")
        print(df_house_price[col].value_counts(normalize=True) , "\n\n")
check_valuecount()

Viewing the unique values in the data with counts in descending order

Id value counts: 
1       0.000343
1940    0.000343
1942    0.000343
1943    0.000343
1944    0.000343
1945    0.000343
1946    0.000343
1947    0.000343
1948    0.000343
1949    0.000343
1950    0.000343
1951    0.000343
1952    0.000343
1953    0.000343
1954    0.000343
1955    0.000343
1956    0.000343
1957    0.000343
1958    0.000343
1959    0.000343
1960    0.000343
1941    0.000343
1939    0.000343
1962    0.000343
1938    0.000343
1919    0.000343
1920    0.000343
1921    0.000343
1922    0.000343
1923    0.000343
1924    0.000343
1925    0.000343
1926    0.000343
1927    0.000343
1928    0.000343
1929    0.000343
1930    0.000343
1931    0.000343
1932    0.000343
1933    0.000343
1934    0.000343
1935    0.000343
1936    0.000343
1937    0.000343
1961    0.000343
1963    0.000343
1826    0.000343
1986    0.000343
1988    0.000343
1989    0.000343
1990    0.000343
1991    0.000343
1992    0.000343
1993    0.

0       0.571429
546     0.007879
728     0.006166
504     0.005824
720     0.004454
600     0.004454
672     0.004454
896     0.003768
886     0.003426
756     0.003083
780     0.003083
862     0.002741
840     0.002398
702     0.002398
601     0.002398
630     0.002055
462     0.002055
739     0.002055
804     0.002055
754     0.002055
689     0.002055
567     0.002055
878     0.002055
744     0.002055
676     0.002055
656     0.001713
550     0.001713
800     0.001713
880     0.001713
741     0.001713
858     0.001713
678     0.001713
668     0.001713
704     0.001713
1142    0.001713
624     0.001713
636     0.001713
684     0.001713
884     0.001713
564     0.001370
828     0.001370
650     0.001370
885     0.001370
864     0.001370
784     0.001370
595     0.001370
644     0.001370
900     0.001370
742     0.001370
378     0.001370
793     0.001370
750     0.001370
596     0.001370
915     0.001370
797     0.001370
850     0.001370
551     0.001370
795     0.001370
640     0.0013

In [88]:
df_house_price['BsmtFullBath'] = df_house_price['BsmtFullBath'].replace([0.0,0],'No')
df_house_price['BsmtFullBath'] = df_house_price['BsmtFullBath'].replace([1.0,2.0,3.0,1,2,3],'Exist_123')
df_house_price['BsmtHalfBath'] = df_house_price['BsmtHalfBath'].replace([0.0,0],'No')
df_house_price['BsmtHalfBath'] = df_house_price['BsmtHalfBath'].replace([1.0,2.0,1,2],'Exist_12')
df_house_price['FullBath'] = df_house_price['FullBath'].replace([0,3,4,1],'Bath_0134')
df_house_price['FullBath'] = df_house_price['FullBath'].replace(2,'Bath_2')
df_house_price['HalfBath'] = df_house_price['HalfBath'].replace(0,'Bath_0')
df_house_price['HalfBath'] = df_house_price['HalfBath'].replace([1,2],'Bath_12')
df_house_price['BedroomAbvGr'] = df_house_price['BedroomAbvGr'].replace([0,1,5,6,8],15680)
df_house_price['KitchenAbvGr'] = df_house_price['KitchenAbvGr'].replace([0,2,3],230)
df_house_price['KitchenQual'] = df_house_price['HeatingQC'].replace(['Gd','Fa','Ex'], 'Not_TA')

df_house_price['TotRmsAbvGrd'] = df_house_price['TotRmsAbvGrd'].replace([1,2,3,4],'RMS_1234')
df_house_price['TotRmsAbvGrd'] = df_house_price['TotRmsAbvGrd'].replace([5,6,7],'RMS_567')
df_house_price['TotRmsAbvGrd'] = df_house_price['TotRmsAbvGrd'].replace([8,9,10,11,12,13,14,15],'RMS_morethan7')
df_house_price['Functional'] = df_house_price['BsmtFinType1'].replace(['Min2','Min1','Mod','Maj1','Maj2','sev'],'No_Typ')
df_house_price['Fireplaces'] = df_house_price['Fireplaces'].replace([2,3,4], '2To4')
df_house_price['FireplaceQu'] = df_house_price['FireplaceQu'].replace(['Gd','Fa','Ex','Po'], 'Not_TA')
df_house_price['GarageType'] = df_house_price['GarageType'].replace(['BuiltIn','Basment','2Types','CarPort'],'Others')
df_house_price['GarageCars'] = df_house_price['GarageCars'].replace([1.0,2.0,1,2],'Exist_12')


df_house_price['HeatingQC'] = df_house_price['HeatingQC'].replace(['Gd','Fa','Po'], 'Not_TA_Ex')

# Replacing values with presence of less than 1 percent by same value/name.
df_house_price['MSSubClass'] = df_house_price['MSSubClass'].replace([50,120,30,160,70,80,90,190,85,75,45,180,40,150],'Others')
df_house_price['MSZoning'] = df_house_price['MSZoning'].replace(['FV','C (all)','RH'], 'Others')
df_house_price['LotShape'] = df_house_price['LotShape'].replace(['IR2','IR3'], 'IR_Others')
df_house_price['LandContour'] = df_house_price['LandContour'].replace(['HLS','Bnk','Low'], 'Others')
df_house_price['LotConfig'] = df_house_price['LotConfig'].replace(['CulDSac','FR2','FR3'], 'Others')

df_house_price['Neighborhood'] = df_house_price['Neighborhood'].replace(['NAmes','CollgCr','OldTown','Edwards','Somerst'], 'Area_1')
df_house_price['Neighborhood'] = df_house_price['Neighborhood'].replace(['NridgHt','Gilbert','Sawyer','NWAmes','SawyerW'], 'Area_2')
df_house_price['Neighborhood'] = df_house_price['Neighborhood'].replace(['Mitchel','BrkSide','Crawfor','IDOTRR','Timber'], 'Area_3')
df_house_price['Neighborhood'] = df_house_price['Neighborhood'].replace(['NoRidge','StoneBr','SWISU','ClearCr','MeadowV','BrDale','Blmngtn','Veenker','NPkVill','Blueste' ], 'Area_4')

df_house_price['Condition1'] = df_house_price['Condition1'].replace(['Feedr','Artery','RRAn','PosN','RRAe','PosA','RRNn','RRNe' ], 'Others')
df_house_price['Condition2'] = df_house_price['Condition2'].replace(['Feedr','Artery','RRAn','PosN','RRAe','PosA','RRNn','RRNe' ], 'Others')

df_house_price['BldgType'] = df_house_price['BldgType'].replace(['TwnhsE','Duplex','Twnhs','2fmCon'], 'Others')
df_house_price['HouseStyle'] = df_house_price['HouseStyle'].replace(['1.5Fin','SLvl','SFoyer','1.5Unf','2.5Unf','2.5Fin' ], 'Others')


df_house_price['OverallQual'] = df_house_price['LotConfig'].replace([1,2,3], 'Qual_1')
df_house_price['OverallQual'] = df_house_price['LotConfig'].replace([4,5,6,7], 'Qual_2')
df_house_price['OverallQual'] = df_house_price['LotConfig'].replace([8,9,10], 'Qual_3')


df_house_price['OverallCond'] = df_house_price['OverallCond'].replace([1,2,3,4,5], 'Qual_1')
df_house_price['OverallCond'] = df_house_price['OverallCond'].replace([4,5,6], 'Qual_2')
df_house_price['OverallCond'] = df_house_price['OverallCond'].replace([7,8,9], 'Qual_3')

df_house_price['RoofStyle'] = df_house_price['RoofStyle'].replace(['Flat','Gambrel','Mansard','Shed'], 'Others')
df_house_price['RoofStyle'] = df_house_price['RoofStyle'].replace(['Hip','Flat','Gambrel','Mansard','Shed'], 'Others')
df_house_price['Exterior1st'] = df_house_price['Exterior1st'].replace(['Plywood','CemntBd','BrkFace','Stone','AsbShng','BrkComm','ImStucc','CBlock','AsphShn','WdShing','Stucco','ImStucc'], 'Others')
df_house_price['Exterior2nd'] = df_house_price['Exterior2nd'].replace(['Plywood','CmentBd','Brk Cmn','Other','Wd Shng','CemntBd','CemntBd','BrkFace','Stone','AsbShng','BrkComm','ImStucc','CBlock','AsphShn','WdShing','Stucco','ImStucc','other'], 'Others')
df_house_price['MasVnrType'] = df_house_price['MasVnrType'].replace(['Stone','BrkCmn'], 'Others')
df_house_price['MasVnrType'] = df_house_price['MasVnrType'].replace('None','No_type')
df_house_price['ExterQual'] = df_house_price['ExterQual'].replace(['Gd','Ex','Fa'], 'Not_TA')
df_house_price['ExterCond'] = df_house_price['ExterCond'].replace(['Gd','Ex','Fa','Po'], 'Not_TA')
df_house_price['Foundation'] = df_house_price['Foundation'].replace(['BrkTil','Slab','Stone','Wood'], 'Others')
df_house_price['BsmtQual'] = df_house_price['BsmtQual'].replace(['Gd','Ex','Fa'], 'Not_TA')
df_house_price['BsmtCond'] = df_house_price['BsmtCond'].replace(['Gd','Ex','Fa','Po'], 'Not_TA')
df_house_price['BsmtExposure'] = df_house_price['BsmtExposure'].replace(['Gd','Mn','Av'], 'Exposure')
df_house_price['BsmtExposure'] = df_house_price['BsmtExposure'].replace('No', 'No_Exposure')
df_house_price['BsmtFinType1'] = df_house_price['BsmtFinType1'].replace(['Rec','BLQ','LwQ'], 'Others')
df_house_price['HeatingQC'] = df_house_price['HeatingQC'].replace(['Gd','Fa','Po'], 'Not_TA_Ex')
df_house_price['Electrical'] = df_house_price['Electrical'].replace(['FuseA','FuseF','FuseP','Mix'], 'Fuse_AFP_Mix')

In [ ]:



df_house_price['LotConfig'] = df_house_price['LotConfig'].replace(['FR2','FR3'], 'FR')
df_house_price['Condition1'] = df_house_price['Condition1'].replace(['RRAe','PosA','RRNn','RRNe'], 'Others')

df_house_price['RoofStyle'] = df_house_price['RoofStyle'].replace(['Flat','Gambrel','Mansard','Shed'], 'Others')
#
df_house_price['Foundation'] = df_house_price['Foundation'].replace(['Stone','Wood'], 'Others')
df_house_price['SaleCondition'] = df_house_price['SaleCondition'].replace(['Alloca','AdjLand'], 'Others')
df_house_price['MasVnrType']=df_house_price['MasVnrType'].replace('None','No_type')

df_house_price['TotRmsAbvGrd'] = df_house_price['TotRmsAbvGrd'].replace([11,3,12,2,14], 'Others')
df_house_price['BedroomAbvGr'] = df_house_price['BedroomAbvGr'].replace([6,0,8], 'Others')
df_house_price['MasVnrType'] = df_house_price['MasVnrType'].replace(['nan','BrkCmn'], 'Others')


df_house_price['FullBath'] = df_house_price['FullBath'].replace([3,0], 'Others')
df_house_price['KitchenQual'] = df_house_price['KitchenQual'].replace(['Ex','Fa'], 'Others')
df_house_price['MSSubClass'] = df_house_price['MSSubClass'].replace(90, 'Others')


In [ ]:
 # Adding total sqfootage feature 
df_house_price['TotalSF'] = df_house_price['TotalBsmtSF'] + df_house_price['1stFlrSF'] + df_house_price['2ndFlrSF']
# Removing TotalBsmtSF,1stFlrSF, 2ndFlrSF and Id
df_house_price=df_house_price.drop(["TotalBsmtSF","1stFlrSF","2ndFlrSF"], axis=1)

#importing the current year
import datetime

currentDateTime = datetime.datetime.now()
date = currentDateTime.date()
current_year = date.strftime("%Y")
current_year=pd.to_numeric(current_year)


#Building new meaningful variables.
df_house_price['house_age'] = current_year - df_house_price['YearBuilt']
df_house_price['house_age_RemodAdd'] = current_year - df_house_price['YearRemodAdd']
df_house_price['house_age_garageYrBlt'] = current_year - df_house_price['GarageYrBlt']
df_house_price['house_age_YrSold'] = current_year - df_house_price['YrSold']

#Dropping the unwanted variables.
df_house_price=df_house_price.drop(columns=['YearBuilt','YearRemodAdd','GarageYrBlt','YrSold'],axis=1)


In [ ]:
def missing_value():
    # Printing total numbers and percentage of missing X_train_num
    total = df.isnull().sum().sort_values(ascending=False)
    percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    print(missing_data)
    

In [ ]:
# if there is less than 70 percent missing values than fill in with median value of column
def missing_value_imputation():
    for col in df.columns.values:
        if df[col].dtype == 'object':
            df.fillna(df[col].mode()[0],inplace=True)

        elif df[col].dtype == 'int' or 'float':
            df.fillna(df[col].median(),inplace=True)

        else:
            pass


In [ ]:
def Outlier_Treatment():
    
    for col in df.select_dtypes(include=['float64','int64']):
        Lower=df[col].quantile(0.25)
        Upper=df[col].quantile(0.75)
        IQR=Upper-Lower
        Lower_Limit=Lower -(1.5*IQR)
        Upper_Limit=Upper+(1.5*IQR)

        #Outliers being imputed with upper and lower limit given by below mention formula.
        df[col] = np.where(df[col] <Lower_Limit,Lower_Limit,df[col])
        df[col] =np.where(df[col] >Upper_Limit, Upper_Limit,df[col])



In [ ]:
df=df_house_price
print("shape of the df_train",df_house_price.shape)
missing_value()
print("shape of the df_train",df_house_price.shape)

In [ ]:

df=df_house_price
print("shape of the df_train",df_house_price.shape)
print('Missing value before imputation',df_house_price.isnull().sum().sum())
missing_value_imputation()
print("shape of the df_train",df_house_price.shape)
print('Missing value after imputation',df_house_price.isnull().sum().sum())


In [ ]:
df_house_price['house_age_garageYrBlt']=df_house_price['house_age_garageYrBlt'].astype('int')

In [ ]:
df=df_house_price
Outlier_Treatment()

In [ ]:
# save all categorical columns in list
categorical_columns = [col for col in df_house_price.columns.values if df_house_price[col].dtype == 'object']

# dataframe with categorical features
df_train_cat = df_house_price[categorical_columns]
# dataframe with numerical features
df_train_num = df_house_price.drop(categorical_columns, axis=1)

In [ ]:
df_train_cat.dtypes

In [ ]:
int(df_house_price['MSSubClass'])

In [ ]:
import math
math.trunc(df_house_price)

In [ ]:
# Replacing values with presence of less than 1 percent by same value/name.
test['MSSubClass'] = test['MSSubClass'].replace([50,120,30,160,70,80,'Others',190,85,75,45,180,40,150],'other')
test['MSZoning'] = test['MSZoning'].replace(['C (all)','RH'], 'Others')

test['LotConfig'] = test['LotConfig'].replace(['FR2','FR3'], 'FR')
test['Condition1'] = test['Condition1'].replace(['RRAe','PosA','RRNn','RRNe'], 'Others')
test['HouseStyle'] = test['HouseStyle'].replace(['1.5Unf','2.5Unf','2.5Fin' ], 'Others')
test['RoofStyle'] = test['RoofStyle'].replace(['Flat','Gambrel','Mansard','Shed'], 'Others')

test['Foundation'] = test['Foundation'].replace(['Stone','Wood'], 'Others')
test['SaleCondition'] = test['SaleCondition'].replace(['Alloca','AdjLand'], 'Others')
test['MasVnrType']=test['MasVnrType'].replace('None','No_type')

test['TotRmsAbvGrd'] = test['TotRmsAbvGrd'].replace([11,3,12,13,15], 'Others')
test['BedroomAbvGr'] = test['BedroomAbvGr'].replace([6,0], 'Others')
test['MasVnrType'] = test['MasVnrType'].replace(['BrkCmn'], 'Others')
test['Fireplaces'] = test['Fireplaces'].replace([3,4], 'Others')

test['FullBath'] = test['FullBath'].replace([3,4,0], 'Others')
test['KitchenQual'] = test['KitchenQual'].replace(['Ex','Fa'], 'Others')




test['Exterior1st'] = test['Exterior1st'].replace(['CemntBd','BrkFace','Others','Stucco','AsbShng'], 'Others')
test['Exterior2nd'] = test['Exterior2nd'].replace(['CemntBd','BrkFace','Others','Stucco','AsbShng','Wd Shng'], 'Other')

In [ ]:
print('Viewing the unique values in the data with counts in descending order')
print()
def check_valuecount():
    for col in test:
        print(col + " value counts: ")
        print(test[col].value_counts() , "\n\n")
check_valuecount()

In [ ]:

scaler=MinMaxScaler()
   
print("shape of the df_train",df_train_num.shape)
df_train_num_drop=df_train_num.drop(['SalePrice'],axis=1)

df_train_scaled=pd.DataFrame(scaler.fit_transform(df_train_num_drop),columns=df_train_num_drop.columns)

print("shape of the df_train",df_train_scaled.shape)


In [ ]:
df_train_scaled.head()

In [ ]:
# Ploting correlation plot
import matplotlib.pyplot as plt
corr=df_train_scaled.corr()
plt.figure(figsize=(30, 30))

sns.heatmap(corr[(corr >= 0.5) | (corr <= -0.5)], 
            cmap='YlGnBu', vmax=1.0, vmin=-1.0, linewidths=0.1,
            annot=True, annot_kws={"size": 8}, square=True);
plt.title('Correlation between features')
print()
print('Findings:')
print()
print('few Plotted variables are showing moderate to high positive correlation. and will be eliminated during VIF check.')
print()


In [ ]:
df_train_final=pd.concat([df_train_scaled,df_train_dummy],axis=1)

In [ ]:
df_train_final.head()

In [ ]:
#Separating target avriable and predictor variables
y_train=df_train_num.pop('SalePrice')
X_train=df_train_final

In [ ]:
X_train.isnull().sum()

# Section5: Model Building


In [ ]:
# Lets build Linear Regression model using statsmodel
import statsmodels.api as sm

# Building Linear Regression model using OLS 
model1 = sm.OLS(y_train, df_train_scaled).fit()
# Note the Swap of X and Y 
# model1.fit()

In [ ]:
# Printing Linear Regression Summary
model1.summary()

In [ ]:
RandomForest_predictions=model1.predict(df_train_scaled)
y_test=y_train

In [ ]:
from sklearn import metrics
from sklearn.metrics import mean_squared_error,mean_absolute_error,explained_variance_score, r2_score

In [ ]:
print('Mean Absolute Error(MAE):', metrics.mean_absolute_error(y_test, RandomForest_predictions))
print('Mean Squared Error(MSE):', metrics.mean_squared_error(y_test, RandomForest_predictions))
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(y_test, RandomForest_predictions)))
print('Explained Variance Score (EVS):',explained_variance_score(y_test,RandomForest_predictions))
print('R2:',metrics.r2_score(y_test, RandomForest_predictions))
print('R2 rounded:',(metrics.r2_score(y_test, RandomForest_predictions)).round(2))
r2 = r2_score(y_test, RandomForest_predictions)
r2_rounded = r2_score(y_test, RandomForest_predictions).round(2)

# Test

In [ ]:
df_test=pd.read_csv('test.csv')
test=df_test.copy()

In [ ]:
 # Adding total sqfootage feature 
test['TotalSF'] = test['TotalBsmtSF'] + test['1stFlrSF'] + test['2ndFlrSF']
# Removing TotalBsmtSF,1stFlrSF, 2ndFlrSF and Id
test=test.drop(["TotalBsmtSF","1stFlrSF","2ndFlrSF"], axis=1)

#importing the current year
import datetime

currentDateTime = datetime.datetime.now()
date = currentDateTime.date()
current_year = date.strftime("%Y")
current_year=pd.to_numeric(current_year)


#Building new meaningful variables.
test['house_age'] = current_year - test['YearBuilt']
test['house_age_RemodAdd'] = current_year - test['YearRemodAdd']
test['house_age_garageYrBlt'] = current_year - test['GarageYrBlt']
test['house_age_YrSold'] = current_year - test['YrSold']

#Dropping the unwanted variables.
test=test.drop(columns=['YearBuilt','YearRemodAdd','GarageYrBlt','YrSold'],axis=1)

In [ ]:
house=df_house_price
house=house.drop(['SalePrice'],axis=1)
test=test[house.columns]

In [ ]:
df=test
missing_value()

In [ ]:
missing_value_imputation()

In [ ]:
Outlier_Treatment()

In [ ]:
# save all categorical columns in list
categorical_columns_test = [col for col in test.columns.values if test[col].dtype == 'object']

# dataframe with categorical features
df_test_cat = test[categorical_columns]
# dataframe with numerical features
df_test_num = test.drop(categorical_columns, axis=1)

In [ ]:
df_train_dummy.shape

In [ ]:
df_train_dummy.head(5)

In [ ]:
df_test_dummy=pd.get_dummies(df_test_cat,drop_first=True)


In [ ]:
df_test_dummy.shape

In [ ]:
set(df_test_dummy).symmetric_difference(set(df_train_dummy))

In [ ]:
df_test_num_scaled=pd.DataFrame(scaler.transform(df_test_num),columns=df_test_num.columns)

In [ ]:
df_house_price['Exterior2nd'].value_counts()

In [ ]:
test['Exterior2nd'].value_counts()

In [ ]:
_AsbShng'

In [ ]:
set(df_train_dummy).symmetric_difference(set(df_test_dummy))

In [ ]:
df_train_dummy['TotRmsAbvGrd_2']

In [ ]:
col=['BedroomAbvGr_8',
 'Condition1_Others',
 'Condition1_PosA',
 'Condition1_RRAe',
 'Condition1_RRNe',
 'Condition1_RRNn',
 'Exterior1st_AsbShng',
 'Exterior1st_AsphShn',
 'Exterior1st_BrkComm',
 'Exterior1st_CBlock',
 'Exterior1st_Others',
 'Exterior1st_WdShing',
 'Exterior2nd_AsbShng',
 'Exterior2nd_AsphShn',
 'Exterior2nd_Brk Cmn',
 'Exterior2nd_CBlock',
 'Exterior2nd_ImStucc',
 'Exterior2nd_Others',
 'Exterior2nd_Stone',
 'Fireplaces_4',
 'Foundation_Others',
 'Foundation_Stone',
 'Foundation_Wood',
 'FullBath_4',
 'GarageCars_1',
 'GarageCars_1.0',
 'GarageCars_2',
 'GarageCars_2.0',
 'GarageCars_3',
 'GarageCars_3.0',
 'GarageCars_4',
 'GarageCars_4.0',
 'GarageCars_5.0',
 'GarageCars_nan',
 'HouseStyle_1.5Unf',
 'HouseStyle_2.5Unf',
 'HouseStyle_Others',
 'KitchenQual_Ex',
 'LotConfig_FR',
 'LotConfig_FR2',
 'LotConfig_FR3',
 'MSSubClass_120',
 'MSSubClass_150',
 'MSSubClass_180',
 'MSSubClass_190',
 'MSSubClass_40',
 'MSSubClass_45',
 'MSSubClass_75',
 'MSSubClass_85',
 'MSZoning_C (all)',
 'MasVnrType_BrkCmn',
 'MasVnrType_No_type',
 'MasVnrType_None',
 'MasVnrType_nan',
 'MoSold',
 'MoSold_Aug',
 'MoSold_Dec',
 'MoSold_Feb',
 'MoSold_Jan',
 'MoSold_July',
 'MoSold_June',
 'MoSold_Mar',
 'MoSold_May',
 'MoSold_Nov',
 'MoSold_Oct',
 'MoSold_Sept',
 'OverallCond_2',
 'OverallCond_3',
 'OverallCond_4',
 'OverallCond_5',
 'OverallCond_6',
 'OverallCond_7',
 'OverallCond_8',
 'OverallCond_9',
 'OverallCond_exce_veryexce',
 'OverallCond_fair_belowAvg',
 'OverallCond_good_verygood',
 'OverallCond_verypoor_poor',
 'OverallQual_10',
 'OverallQual_2',
 'OverallQual_3',
 'OverallQual_4',
 'OverallQual_5',
 'OverallQual_6',
 'OverallQual_7',
 'OverallQual_8',
 'OverallQual_9',
 'OverallQual_exce_veryexce',
 'OverallQual_fair_belowAvg',
 'OverallQual_good_verygood',
 'OverallQual_verypoor_poor',
 'RoofStyle_Gable',
 'RoofStyle_Gambrel',
 'RoofStyle_Mansard',
 'RoofStyle_Others',
 'RoofStyle_Shed',
 'SaleCondition_AdjLand',
 'SaleCondition_Alloca',
 'SaleCondition_Others',
 'TotRmsAbvGrd_13',
 'TotRmsAbvGrd_14',
 'TotRmsAbvGrd_15','TotRmsAbvGrd_2']

In [ ]:
df_test_dummy=df_test_dummy.drop(col,axis=1)

# MAIN TEST SET

In [ ]:
#make the prediction
y_pred = model1.predict(df_test_num_scaled)
y_pred[:5]

In [ ]:
df = pd.DataFrame(y_pred,columns=['SalePrice'])
df.head()

In [ ]:
sol=pd.read_csv('sample_submission.csv')
sol.head()

In [ ]:
sol['SalePrice']=df['SalePrice']
sol.head()

In [ ]:
sol.to_csv('submissionHS.csv',index=False)